In [1]:
import torch
import numpy as np
import pandas as pd
import matchzoo as mz
print('matchzoo version', mz.__version__)

matchzoo version 1.2


In [2]:
ranking_task = mz.tasks.Ranking(losses=mz.losses.RankHingeLoss())
ranking_task.metrics = [
    mz.metrics.NormalizedDiscountedCumulativeGain(k=3),
    mz.metrics.NormalizedDiscountedCumulativeGain(k=5),
    mz.metrics.MeanAveragePrecision()
]
print("`ranking_task` initialized with metrics", ranking_task.metrics)

`ranking_task` initialized with metrics [normalized_discounted_cumulative_gain@3(0.0), normalized_discounted_cumulative_gain@5(0.0), mean_average_precision(0.0)]


In [3]:
print('data loading ...')
train_pack_raw = mz.datasets.wiki_qa.load_data('train', task=ranking_task)
dev_pack_raw = mz.datasets.wiki_qa.load_data('dev', task=ranking_task, filtered=True)
test_pack_raw = mz.datasets.wiki_qa.load_data('test', task=ranking_task, filtered=True)
print('data loaded as `train_pack_raw` `dev_pack_raw` `test_pack_raw`')

data loading ...
data loaded as `train_pack_raw` `dev_pack_raw` `test_pack_raw`


In [4]:
preprocessor = mz.models.DUET.get_default_preprocessor(
    filter_mode='df',
    filter_low_freq=2,
    truncated_mode='post',
    truncated_length_left=10,
    truncated_length_right=40,
    ngram_size=3
)

In [5]:
train_pack_processed = preprocessor.fit_transform(train_pack_raw)
dev_pack_processed = preprocessor.transform(dev_pack_raw)
test_pack_processed = preprocessor.transform(test_pack_raw)

Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 2118/2118 [00:00<00:00, 5194.80it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 18841/18841 [00:06<00:00, 2940.48it/s]
Building Vocabulary from a datapack.: 100%|██████████| 1976372/1976372 [00:00<00:00, 2263367.08it/s]
Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 2118/2118 [00:00<00:00, 6913.96it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 18841/18841 [00:06<00:00, 2745.74it/s]
Processing length_right with len: 100%|██████████| 18841/18841 [00:00<00:00, 555219.04it/s]
Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 122/122 [00:00<00:00, 7671.40it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 1115/1115 [00:00<

In [6]:
preprocessor.context

{'ngram_process_unit': <matchzoo.preprocessors.units.ngram_letter.NgramLetter at 0x7f495ff71860>,
 'filter_unit': <matchzoo.preprocessors.units.frequency_filter.FrequencyFilter at 0x7f495ff71e10>,
 'vocab_unit': <matchzoo.preprocessors.units.vocabulary.Vocabulary at 0x7f490a4e4588>,
 'vocab_size': 16675,
 'embedding_input_dim': 16675,
 'ngram_vocab_unit': <matchzoo.preprocessors.units.vocabulary.Vocabulary at 0x7f48fbd5cb00>,
 'ngram_vocab_size': 6602}

In [7]:
triletter_callback = mz.dataloader.callbacks.Ngram(preprocessor, mode='sum')
trainset = mz.dataloader.Dataset(
    data_pack=train_pack_processed,
    mode='pair',
    num_dup=2,
    num_neg=1,
    batch_size=20,
    resample=True,
    sort=False,
    callbacks=[triletter_callback]
)
testset = mz.dataloader.Dataset(
    data_pack=test_pack_processed,
    batch_size=20,
    callbacks=[triletter_callback]
)

In [8]:
padding_callback = mz.models.DUET.get_default_padding_callback(
    fixed_length_left=10,
    fixed_length_right=40,
    pad_word_value=0,
    pad_word_mode='pre',
    with_ngram=True
)

trainloader = mz.dataloader.DataLoader(
    dataset=trainset,
    stage='train',
    callback=padding_callback
)
testloader = mz.dataloader.DataLoader(
    dataset=testset,
    stage='dev',
    callback=padding_callback
)

In [9]:
model = mz.models.DUET()

model.params['task'] = ranking_task
model.params['left_length'] = 10
model.params['right_length'] = 40
model.params['lm_filters'] = 100
model.params['mlp_num_layers'] = 2
model.params['mlp_num_units'] = 100
model.params['mlp_num_fan_out'] = 100
model.params['mlp_activation_func'] = 'tanh'

model.params['vocab_size'] = preprocessor.context['ngram_vocab_size']
model.params['dm_conv_activation_func'] = 'relu'
model.params['dm_filters'] = 100
model.params['dm_kernel_size'] = 3
model.params['dm_right_pool_size'] = 4
model.params['dropout_rate'] = 0.2


model.build()

print(model)
print('Trainable params: ', sum(p.numel() for p in model.parameters() if p.requires_grad))

DUET(
  (lm_conv1d): Conv1d(40, 100, kernel_size=(1,), stride=(1,))
  (lm_mlp): Sequential(
    (0): Sequential(
      (0): Linear(in_features=1000, out_features=100, bias=True)
      (1): Tanh()
    )
    (1): Sequential(
      (0): Linear(in_features=100, out_features=100, bias=True)
      (1): Tanh()
    )
    (2): Sequential(
      (0): Linear(in_features=100, out_features=100, bias=True)
      (1): Tanh()
    )
  )
  (lm_linear): Sequential(
    (0): Linear(in_features=100, out_features=1, bias=True)
    (1): ReLU()
  )
  (dm_conv_activation_func): ReLU()
  (dm_conv_left): Conv1d(6602, 100, kernel_size=(3,), stride=(1,))
  (dm_mlp_left): Sequential(
    (0): Linear(in_features=100, out_features=100, bias=True)
    (1): ReLU()
  )
  (dm_conv1_right): Conv1d(6602, 100, kernel_size=(3,), stride=(1,))
  (dm_conv2_right): Conv1d(100, 100, kernel_size=(1,), stride=(1,))
  (dm_mlp): Sequential(
    (0): Sequential(
      (0): Linear(in_features=900, out_features=100, bias=True)
      (1)

In [10]:
optimizer = torch.optim.Adadelta(model.parameters())

trainer = mz.trainers.Trainer(
    model=model,
    optimizer=optimizer,
    trainloader=trainloader,
    validloader=testloader,
    validate_interval=None,
    epochs=10
)

In [ ]:
trainer.run()

[Iter-102 Loss-0.749]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.336 - normalized_discounted_cumulative_gain@5(0.0): 0.4315 - mean_average_precision(0.0): 0.3891



[Iter-204 Loss-0.645]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.3329 - normalized_discounted_cumulative_gain@5(0.0): 0.4023 - mean_average_precision(0.0): 0.3736



[Iter-306 Loss-0.613]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.3328 - normalized_discounted_cumulative_gain@5(0.0): 0.4295 - mean_average_precision(0.0): 0.3885



[Iter-408 Loss-0.594]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.3582 - normalized_discounted_cumulative_gain@5(0.0): 0.4451 - mean_average_precision(0.0): 0.4076



[Iter-510 Loss-0.588]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.3342 - normalized_discounted_cumulative_gain@5(0.0): 0.415 - mean_average_precision(0.0): 0.3777



[Iter-612 Loss-0.568]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.3461 - normalized_discounted_cumulative_gain@5(0.0): 0.4219 - mean_average_precision(0.0): 0.3943



[Iter-714 Loss-0.544]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.3712 - normalized_discounted_cumulative_gain@5(0.0): 0.4409 - mean_average_precision(0.0): 0.4108



[Iter-816 Loss-0.558]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.357 - normalized_discounted_cumulative_gain@5(0.0): 0.4435 - mean_average_precision(0.0): 0.4053



[Iter-918 Loss-0.529]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.3201 - normalized_discounted_cumulative_gain@5(0.0): 0.4048 - mean_average_precision(0.0): 0.3648

